In [41]:
# Import des bibliothèques principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [ ]:
# Charger les données
hr_data = pd.read_csv('general_data.csv')
survey_data = pd.read_csv('employee_survey_data.csv')
manager_data = pd.read_csv('manager_survey_data.csv')
in_time_data = pd.read_csv('in_time.csv')
out_time_data = pd.read_csv('out_time.csv')

# Aperçu des données
print(in_time_data.head())
print(out_time_data.head())


In [ ]:
# Renommer la première colonne pour qu'elle devienne "EmployeeID" dans in_time et out_time
in_time_data.rename(columns={"Unnamed: 0": 'EmployeeID'}, inplace=True)
out_time_data.rename(columns={"Unnamed: 0": 'EmployeeID'}, inplace=True)

# Vérifier les modifications
print(in_time_data.head())

print(out_time_data.head())

# Sauvegarder les fichiers modifiés si nécessaire
in_time_data.to_csv('in_time_modified.csv', index=False)
out_time_data.to_csv('out_time_modified.csv', index=False)

# Charger les fichiers modifiés
in_time_data = pd.read_csv('in_time_modified.csv')
out_time_data = pd.read_csv('out_time_modified.csv')

# Fusionner les données de chaque fichier
hr_data = hr_data.merge(survey_data, on='EmployeeID')
hr_data = hr_data.merge(manager_data, on='EmployeeID')
hr_data = hr_data.merge(in_time_data, on='EmployeeID')
hr_data = hr_data.merge(out_time_data, on='EmployeeID')

# Aperçu des données fusionnées
print(hr_data.head())

